In [1]:
from pydantic import BaseModel
from pydantic import BaseModel, Field

from typing import List, Union, Any



In [2]:
class Attribute(BaseModel):
    name: str = Field(
        ...,
        description="Name of the attribute, representing a specific characteristic or property of an entity."
    )
    value: Any = Field(
        ...,
        description="Value of the attribute, which can be of any data type, representing the state or property value of the entity."
    )

class Entity(BaseModel):
    id: int = Field(
        ...,
        description="Unique identifier for the entity, used for tracking and referencing."
    )
    name: str = Field(
        ...,
        description="Name of the entity, representing an object or concept within the GOAP framework."
    )
    attributes: List[Attribute] = Field(
        ...,
        description="List of attributes of the entity, detailing the characteristics and properties associated with the entity."
    )
    time_step: int = Field(
        ...,
        description="Time step at which the entity is relevant, allowing for modeling of entities that vary over time."
    )

class Action(BaseModel):
    name: str = Field(
        ...,
        description="Name of the action, describing the interaction or event."
    )
    time_step: int = Field(
        ...,
        description="Time step at which the action takes place."
    )
    source_entity_ids: List[int] = Field(
        ...,
        description="List of IDs of the source entities initiating the action."
    )
    target_entity_ids: List[int] = Field(
        ...,
        description="List of IDs of the target entities affected by the action. If the action is reflexive, the source and target entities are the same."
    )
    description: str = Field(
        ...,
        description="Description of the action, providing context and details about the interaction."
    )

class TimeStepEntity(BaseModel):
    time_step: int = Field(
        ...,
        description="Time step at which the entities' states and actions are relevant."
    )
    entities: List[Entity] = Field(
        ...,
        description="List of entities present at this time step, each with their respective attributes."
    )
    actions: List[Action] = Field(
        ...,
        description="List of actions occurring at this time step, involving the entities."
    )

class EntitiesExtraction(BaseModel):
    time_step_entities: List[TimeStepEntity] = Field(
        ...,
        description="A list capturing entities, their attributes, and actions across different time steps."
    )



system_prompt = """
Analyze the given text and extract entities, their attributes, and actions occurring at each time step. Populate the TimeStepEntity class with this information. The text to analyze is: "<INSERT TEXT HERE>". Follow these steps:

1. **Identify Entities and Attributes**: For each time step in the text, identify all the entities present and their attributes. Assign each entity a unique ID for reference.

2. **Extract Actions**: Identify any actions that occur at each time step. These actions should be associated with the entities involved, either as initiators (source) or receivers (target) of the action.

3. **Populate TimeStepEntity**: For each time step identified, create a TimeStepEntity object that includes both the list of entities (with their attributes) and the list of actions occurring at that time.

4. **Sequential Organization**: Organize the TimeStepEntity objects sequentially, starting from the earliest time step in the narrative.

Your goal is to create a structured representation of the narrative, capturing the dynamic interplay of entities and actions over time.

Example of a correctly formatted response for a given time step:

```python
TimeStepEntity(
    time_step=0,
    entities=[
        Entity(id=1, name="Entity1", attributes=[Attribute(name="Attribute1", value="Value1")]),
        Entity(id=2, name="Entity2", attributes=[Attribute(name="Attribute2", value="Value2")])
    ],
    actions=[
        Action(name="Action1", time_step=0, source_entity_ids=[1], target_entity_ids=[2], description="Description of Action1")
    ]
)
"""


In [3]:
from graphviz import Digraph
from typing import List, Any

# Assuming the classes Entity, Attribute, Action, and TimeStepEntity are defined as provided

def generate_html_label(entity: Entity) -> str:
    rows = [f"<tr><td>{attr.name}</td><td>{attr.value}</td></tr>" for attr in entity.attributes]
    table_rows = "".join(rows)
    return f"<<table border='0' cellborder='1' cellspacing='0'><tr><td colspan='2'><b>{entity.name}</b></td></tr>{table_rows}</table>>"

def generate_static_graph(data: EntitiesExtraction):
    dot = Digraph(comment="Entity Dynamics Graph", node_attr={"shape": "plaintext"})

    # Create nodes for each entity with their attributes
    for time_step_entity in data.time_step_entities:
        for entity in time_step_entity.entities:
            label = generate_html_label(entity)
            dot.node(str(entity.id), label)

    # Create edges for actions
    for time_step_entity in data.time_step_entities:
        for action in time_step_entity.actions:
            for source_id in action.source_entity_ids:
                for target_id in action.target_entity_ids:
                    dot.edge(str(source_id), str(target_id), label=f"{action.name} (Time Step {action.time_step})\n{action.description}")

    dot.render("entity_dynamics.gv", view=True)


def generate_graph_horizontal(data: EntitiesExtraction, prompt: str):
    dot = Digraph(comment="State Action Graph", node_attr={"shape": "plaintext"})
    dot.attr(labelloc='t')
    dot.attr(label='State Action Graph')

    # Creating clusters for each time step
    for time_step_entity in data.time_step_entities:
        with dot.subgraph(name=f'cluster_{time_step_entity.time_step}') as cluster:
            cluster.attr(label=f'Time Step {time_step_entity.time_step}')
            cluster.attr(style='filled')
            cluster.attr(color='lightgrey')

            # Create nodes for each entity within the time step
            for entity in time_step_entity.entities:
                label = generate_html_label(entity)
                cluster.node(f'entity_{entity.id}_{time_step_entity.time_step}', label)

            # Create edges for actions within the time step
            for action in time_step_entity.actions:
                for source_id in action.source_entity_ids:
                    for target_id in action.target_entity_ids:
                        cluster.edge(f'entity_{source_id}_{time_step_entity.time_step}', 
                                     f'entity_{target_id}_{time_step_entity.time_step}', 
                                     label=f"{action.name}\n{action.description}")

    # Adding the prompt at the bottom of the graph
    dot.attr(labeljust='b', label=prompt)

    dot.render("state_action_graph.gv", view=True)

def generate_graph_vertical(data: EntitiesExtraction, prompt: str):
    dot = Digraph(comment="State Action Graph", node_attr={"shape": "plaintext"})
    dot.attr(rankdir='TB')  # Top to Bottom layout

    # Invisible node for alignment
    dot.node("start", style="invisible")

    # Creating clusters for each time step
    for idx, time_step_entity in enumerate(data.time_step_entities, start=1):
        with dot.subgraph(name=f'cluster_{idx}') as cluster:
            cluster.attr(label=f'Time Step {time_step_entity.time_step}')
            cluster.attr(style='filled', color='lightgrey')

            # Invisible node for internal cluster alignment
            cluster.node(f'invisible_{idx}', style="invisible")

            # Create nodes for each entity within the time step
            for entity in time_step_entity.entities:
                label = generate_html_label(entity)
                cluster.node(f'entity_{entity.id}_{idx}', label)

            # Connect the invisible nodes to center-align the clusters
            if idx > 1:  # Skip the first cluster
                dot.edge(f'invisible_{idx-1}', f'invisible_{idx}', style='invis')

            # Create edges for actions within the time step
            for action in time_step_entity.actions:
                for source_id in action.source_entity_ids:
                    for target_id in action.target_entity_ids:
                        source_node_name = f'entity_{source_id}_{idx}'
                        target_node_name = f'entity_{target_id}_{idx}'
                        cluster.edge(source_node_name, target_node_name, label=f"{action.name}\n{action.description}")

    # Edge from start node to first cluster's invisible node
    dot.edge("start", "invisible_1", style="invis")

    # Adding the prompt at the bottom of the graph
    dot.attr(labelloc='t', label=prompt)

    dot.render("state_action_graph.gv", view=True)





In [4]:
import instructor
from openai import OpenAI

# Apply the patch to the OpenAI client
# enables response_model keyword
client = instructor.patch(OpenAI(api_key = ""))

def ask_ai(content,base_instructions,extra_instructions,response_model):
    return client.chat.completions.create(
        model="gpt-4-1106-preview",
        response_model=response_model,
        max_retries=3,
        messages=[
            {
                "role": "system",
                "content": "Use this instructions" + base_instructions+   extra_instructions,
            },
            {
                "role": "user",
                "content":  content,
            },
        ],
    )  # type: ignore


In [5]:
tiny_story = "Margaret decided to cook a chocolate cake. She prepared the ingredients and mixed them together. She then put the cake in the oven and waited for it to bake. After 30 minutes, she took the cake out of the oven and let it cool. She then decorated the cake with icing and sprinkles. Margaret was very happy with how the cake turned out."
response_tiny_story = ask_ai(tiny_story,system_prompt,"remember to characterize each entity by their attributes, and be sure that causally relevant actions always intervene over some measured attribute of the target entity. Remember that each entity can take as single action for each time step",EntitiesExtraction)

In [6]:
generate_graph_vertical(response_tiny_story,tiny_story)


In [7]:
dnd = """ A mage, robed in flowing, rune-etched garments, stands amidst a clearing surrounded by ancient, towering trees. Their staff, crowned with a glowing crystal, channels the raw energies of the arcane. The air crackles with magical power as the mage prepares to cast a spell, their eyes glowing with a supernatural light.

Suddenly, from the shadows, a rogue emerges, cloaked in a hooded garb that blends seamlessly with the dark forest. The rogue moves with an almost supernatural grace, a pair of gleaming daggers in hand, reflecting the faint light of the moon. They are the embodiment of agility and stealth, their eyes fixed intently on the mage, calculating every possible outcome in this high-stakes confrontation.

As the mage begins to chant, arcs of electricity dancing between their fingers, the rogue dashes forward with incredible speed. The mage, sensing the imminent attack, conjures a shield of shimmering energy, just as the rogue's dagger strikes. Sparks fly as steel meets magical barrier. The rogue, undeterred, rolls back and throws a series of small, enchanted bombs that explode in a cloud of smoke and debris, aiming to disorient the mage. """

In [8]:
response_dnd = ask_ai(dnd,system_prompt,"remember to characterize each entity by their attributes, and be sure that causally relevant actions always intervene over some measured attribute of the target entity. Remember that each entity can take as single action for each time step",EntitiesExtraction)

In [9]:
generate_graph_vertical(response_dnd,dnd)


In [10]:
vox_machina = """ Pike grew up in the outskirts of town, near the Bramblewood.
Her ancestors were a family of deep gnomes with quite an unfavorable reputation.
Thievery, destruction, and trickery left them with the curse of the last name Trickfoot.
Sarenrae, the goddess of healing and redemption, had other plans for Pike's great-great-grandfather, Wilhand, who left his family at a young age after a dream.
A dream that changed the course of the Trickfoot family.
Wilhand devoted his life to Sarenrae and pledged from then on that him and his family would live a life of service and devotion.
As a child, Pike seemed to have an affinity to heal.
Whether it was animals, people, or even flowers, she felt she had a purpose in making things whole that had once been broken.
She studied and learned the ways to heal through divine magic.
She lived a peaceful life, quiet and simple, until one day, Wilhand was captured and almost killed by a group of goliath barbarians.
One of the goliaths took a stand against the murder of the innocent gnome and he himself was beaten, bloodied, and left for dead, abandoned by his herd.
Wilhand went to Pike for help.
She prayed and healed this barbarian as best she could, bringing him back to life.
When he awoke, she discovered his name was Grog Strongjaw.
After that, they were best of friends, a rather unlikely pair.
Little did she know that in a few years' time Grog would soon return the favor and bring her back from the clutches of death.
After being killed in battle, Pike felt angry.
She wanted to be stronger so that it would never happen again.
She spent four months at sea training with the men and women aboard a ship called the Broken Howl.
Gripping her holy symbol in one hand, and her morningstar in the other, this time, Pike is ready. """

In [11]:
response_vox = ask_ai(vox_machina,system_prompt,"remember to characterize each entity by their attributes, and be sure that causally relevant actions always intervene over some measured attribute of the target entity. Remember that each entity can take as single action for each time step",EntitiesExtraction)
generate_graph_vertical(response_vox,vox_machina)


In [12]:
polars = """ Getting Started With Polars DataFrames
Like most other data processing libraries, the core data structure used in Polars is the DataFrame. A DataFrame is a two-dimensional data structure composed of rows and columns. The columns of a DataFrame are made up of series, which are one-dimensional labeled arrays.

You can create a Polars DataFrame in a few lines of code. In the following example, you’ll create a Polars DataFrame from a dictionary of randomly generated data representing information about houses. Be sure you have NumPy installed before running this example:

>>> import numpy as np
>>> import polars as pl

>>> num_rows = 5000
>>> rng = np.random.default_rng(seed=7)

>>> buildings_data = {
...      "sqft": rng.exponential(scale=1000, size=num_rows),
...      "year": rng.integers(low=1995, high=2023, size=num_rows),
...      "building_type": rng.choice(["A", "B", "C"], size=num_rows),
...  }
>>> buildings = pl.DataFrame(buildings_data)
>>> buildings
shape: (5_000, 3)
┌─────────────┬──────┬───────────────┐
│ sqft        ┆ year ┆ building_type │
│ ---         ┆ ---  ┆ ---           │
│ f64         ┆ i64  ┆ str           │
╞═════════════╪══════╪═══════════════╡
│ 707.529256  ┆ 1996 ┆ C             │
│ 1025.203348 ┆ 2020 ┆ C             │
│ 568.548657  ┆ 2012 ┆ A             │
│ 895.109864  ┆ 2000 ┆ A             │
│ …           ┆ …    ┆ …             │
│ 408.872783  ┆ 2009 ┆ C             │
│ 57.562059   ┆ 2019 ┆ C             │
│ 3728.088949 ┆ 2020 ┆ C             │
│ 686.678345  ┆ 2011 ┆ C             │
└─────────────┴──────┴───────────────┘
In this example, you first import numpy and polars with aliases of np and pl, respectively. Next, you define num_rows, which determines how many rows will be in the randomly generated data. To generate random numbers, you call default_rng() from NumPy’s random module. This returns a generator that can produce a variety of random numbers according to different probability distributions.

You then define a dictionary with the entries sqft, year, and building_type, which are randomly generated arrays of length num_rows. The sqft array contains floats, year contains integers, and the building_type array contains strings. These will become the three columns of a Polars DataFrame.

"""

In [14]:
response_polar = ask_ai(polars,system_prompt,"remember to characterize each entity by their attributes, and be sure that causally relevant actions always intervene over some measured attribute of the target entity. Remember that each entity can take as single action for each time step",EntitiesExtraction)
generate_graph_vertical(response_polar,polars)